<a href="https://colab.research.google.com/github/tricksterwink/btp/blob/main/inceptionv3_trail03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing necessary libraries

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.utils import to_categorical

In [ ]:
# Define the training and validation data directories
train_dir = '/content/drive/MyDrive/Sample Rice/Train'

In [ ]:
# Initialize an ImageDataGenerator for preprocessing with augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
# Use the datagen to generate augmented training data
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(512, 512),
    batch_size=64,
    class_mode='categorical',
    subset='training',
    seed=123
)

Found 784 images belonging to 2 classes.


In [ ]:
# Use a separate generator for validation data without augmentation
val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(512, 512),
    batch_size=64,
    class_mode='categorical',
    subset='validation',
    seed=123
)

Found 196 images belonging to 2 classes.


In [ ]:
# Define a custom learning rate schedule function
def learning_rate_schedule(epoch):
    initial_learning_rate = 0.0001
    decay = 0.9
    min_learning_rate = 1e-7

    new_learning_rate = initial_learning_rate * (decay ** epoch)
    new_learning_rate = max(new_learning_rate, min_learning_rate)

    return new_learning_rate

In [ ]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience = 2,
    restore_best_weights = True
)

In [ ]:
# Load ResNet50 without the top classification layers
pretrained_model = tf.keras.applications.InceptionV3(include_top = False,
                   input_shape = (512, 512, 3),
                   pooling = 'avg',
                   weights = 'imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
# Create your custom classification model
inceptionv3_model = Sequential()
inceptionv3_model.add(pretrained_model)
inceptionv3_model.add(Flatten())

inceptionv3_model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
inceptionv3_model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))

In [ ]:
# output layer
inceptionv3_model.add(Dense(2, activation='softmax'))

In [ ]:
inceptionv3_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                                 
Total params: 22343906 (85.24 MB)
Trainable params: 541122 (2.06 MB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________


In [ ]:
# compile the model
inceptionv3_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Use a LearningRateScheduler callback during model training
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(learning_rate_schedule)

epochs = 10
history = inceptionv3_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/10
13/13 [==============================] - 1287s 96s/step - loss: 1.1699 - accuracy: 0.6671 - val_loss: 1.0650 - val_accuracy: 0.8878 - lr: 1.0000e-04
Epoch 2/10
13/13 [==============================] - 1109s 84s/step - loss: 0.9974 - accuracy: 0.9503 - val_loss: 0.9324 - val_accuracy: 0.9337 - lr: 9.0000e-05
Epoch 3/10
13/13 [==============================] - 1106s 84s/step - loss: 0.8881 - accuracy: 0.9719 - val_loss: 0.8402 - val_accuracy: 0.9541 - lr: 8.1000e-05
Epoch 4/10
13/13 [==============================] - 1072s 81s/step - loss: 0.8127 - accuracy: 0.9681 - val_loss: 0.7816 - val_accuracy: 0.9694 - lr: 7.2900e-05
Epoch 5/10
13/13 [==============================] - 1125s 86s/step - loss: 0.7490 - accuracy: 0.9821 - val_loss: 0.7221 - val_accuracy: 0.9796 - lr: 6.5610e-05
Epoch 6/10
13/13 [==============================] - 1111s 91s/step - loss: 0.7097 - accuracy: 0.9758 - val_loss: 0.7001 - val_accuracy: 0.9694 - lr: 5.9049e-05
Epoch 7/10
13/13 [======================

In [ ]:
# Save the model
inceptionv3_model.save('/content/drive/MyDrive/Sample Rice/inceptionv3_model_trail03.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# load saved model
from keras.models import load_model

loaded_model = load_model('/content/drive/MyDrive/Sample Rice/inceptionv3_model_trail03.h5')

In [ ]:
# predicting
from keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = '//content/drive/MyDrive/Sample Rice/Test/1121RAW/112100101.JPG'
img = image.load_img(img_path, target_size=(512, 512))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Make prediction
predictions = loaded_model.predict(x)

# The predictions will be a probability distribution over the classes
# using argmax to get the predicted class
predicted_class = np.argmax(predictions[0])
print(predicted_class)

1/1 [==============================] - 2s 2s/step
0


In [ ]:
# Load and preprocess the image
img_path = '/content/drive/MyDrive/Sample Rice/Test/PR14RAW/PR14RAW00101.JPG'
img = image.load_img(img_path, target_size=(512, 512))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Make prediction
predictions = loaded_model.predict(x)

# The predictions will be a probability distribution over the classes
# using argmax to get the predicted class
predicted_class = np.argmax(predictions[0])
print(predicted_class)

1/1 [==============================] - 1s 680ms/step
1


In [ ]:
import os

# Define the directory containing the test images
test_dir = '/content/drive/MyDrive/Sample Rice/Test'

# Get the list of subdirectories (which are the classes)
classes = os.listdir(test_dir)

for class_name in classes:
    class_dir = os.path.join(test_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(512, 512))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x / 255.0

        # Make prediction
        predictions = loaded_model.predict(x)

        # If you have binary classification, you can use argmax to get the predicted class
        predicted_class = np.argmax(predictions[0])

        print(f"Image '{img_name}' in class '{class_name}' is predicted as class {predicted_class}")

1/1 [==============================] - 1s 934ms/step
Image '112100101.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 651ms/step
Image '112100102.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 652ms/step
Image '112100103.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 659ms/step
Image '112100104.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 662ms/step
Image '112100105.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 1s/step
Image '112100106.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 1s/step
Image '112100107.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 669ms/step
Image '112100108.JPG' in class '1121RAW' is predicted as class 0
1/1 [==============================] - 1s 660ms/step
Image '11

In [ ]:
import os

# Define the directory containing the test images
test_dir = '/content/drive/MyDrive/Sample Rice/Test'

# Get the list of subdirectories (which are the classes)
classes = os.listdir(test_dir)

# Initialize variables to keep track of total and correct predictions
total_predictions = 0
correct_predictions = 0

for class_name in classes:
    class_dir = os.path.join(test_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(512, 512))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x / 255.0

        # Make prediction
        predictions = loaded_model.predict(x)
        predicted_class = np.argmax(predictions[0])

        if predicted_class == classes.index(class_name):  # Assuming classes are sorted alphabetically
            correct_predictions += 1

        total_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_predictions

print(f"Total predictions: {total_predictions}")
print(f"Correct predictions: {correct_predictions}")
print(f"Accuracy: {accuracy*100:.2f}%")

1/1 [==============================] - 1s 669ms/step
Total predictions: 20
Correct predictions: 19
Accuracy: 95.00%
